# Amortization Calculator Application

This application allows you to view the changes of the behavior of an amortization schedule based on different inputs. Run all of the following cells in order to generate a dashboard which displays both table and graph versions of the amort schedule. You can then move the display of the final output in order to just see the dashboard. 

## Imports

In [1]:
# Imports
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy_financial as npf
from datetime import datetime
import ipysheet
import seaborn as sns

# Imports from helper functions
from AmortCalc.Application import *

## Setup

In [2]:
# Creating the main inputs for the application 

form_item_layout = widgets.Layout(justify_content='space-between')

principal = widgets.IntText(description = "Principal", value = 100000)
period_in_years = widgets.IntSlider(min = 1, max = 60, description = "Years")
period_in_months = widgets.IntSlider(min = 1, description = "Period")
rate = widgets.FloatText(description = "Rate (ex) 2.5)", value = 2, style=dict(description_width='initial'))
start_date = widgets.DatePicker(value = datetime.today())
additional_payment = widgets.IntText(description = "Additional Principal Per Period", value = 50, min = 0,style=dict(description_width='initial'))


# This function changes the max value of the period based on the amount of years inputted
def update_x_range(*args):
    period_in_months.max = 12.0 * period_in_years.value

period_in_months.observe(update_x_range, 'value')
    

## Inputs

In [3]:
ui = widgets.VBox([principal,
                   rate,
                   start_date,
                   period_in_years,
                   period_in_months,
                   additional_payment],layout = form_item_layout)

value_output = widgets.interactive_output(payments_per_period,
                                          {'rate_in_months': rate,
                                           'per': period_in_months,
                                           'period_in_months':period_in_years,
                                           'principal':principal })
inputs = widgets.VBox([value_output,ui],layout = widgets.Layout(border='2px solid gray', padding = '10px',
                                                       align_items = 'stretch', width='38%'));



## Outputs

In [4]:
out = widgets.Output(layout={'border': '1px solid black'})

view_toggle = widgets.Dropdown(
    options=['Table', 'Graph'],
    value='Table',
    description='View:',
    disabled=False,)

def toggle(x):
    if x == 'Table':
        display(widgets.HBox([inputs,table_output]))
    elif x == 'Graph':
        display(widgets.HBox([inputs,graph_output]))
        
table_output = widgets.interactive_output(create_table,
                                      {'start_date': start_date,
                                       'rate_in_months': rate,
                                       'period_in_years': period_in_years,
                                       'principal': principal,
                                      'additional_pmt':additional_payment})
graph_output = widgets.interactive_output(create_graph,
                                      {'start_date': start_date,
                                       'rate_in_months': rate,
                                       'period_in_years': period_in_years,
                                       'principal': principal,
                                      'additional_pmt':additional_payment})
    
with out: 
    outputs = widgets.interact(toggle, x = view_toggle)
out

Output(layout=Layout(border='1px solid black'))

## Dependencies

In [5]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2021-08-16T11:29:37.563937-04:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.4.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

ipysheet       : 0.5.0
matplotlib     : 3.3.4
numpy_financial: 1.0.0
numpy          : 1.20.1
seaborn        : 0.11.1
pandas         : 1.2.4
ipywidgets     : 7.6.3

